In [5]:
sc.getConf().getAll()

[('spark.sql.warehouse.dir',
  'file:/home/rmsryu/notebooks/CA2/ca2-twitter/notebooks/spark-warehouse'),
 ('spark.jars',
  'file:///opt/azure-storage-jars/azure-storage-2.0.0.jar,file:///opt/azure-storage-jars/hadoop-azure-2.7.5.jar,file:///dsvm/tools/spark/spark-3.2.3/jars/mongo-spark-connector_2.13-10.1.1.jar'),
 ('spark.app.initial.jar.urls',
  'spark://rmsryu-vm.internal.cloudapp.net:45523/jars/azure-storage-2.0.0.jar,spark://rmsryu-vm.internal.cloudapp.net:45523/jars/mongo-spark-connector_2.13-10.1.1.jar,spark://rmsryu-vm.internal.cloudapp.net:45523/jars/hadoop-azure-2.7.5.jar'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '5g'),
 ('spark.app.name', 'pyspark-shell'),
 ('spark.driver.port', '45523'),
 ('spark.sql.catalogImplementation', 'hive'),
 ('spark.repl.local.jars',
  'file:///opt/azure-storage-jars/azure-storage-2.0.0.jar,file:///opt/azure-storage-jars/hadoop-azure-2.7.5.jar,file:///dsvm/tools/spark/spark-3.2.3/jars/mongo-spark-connector_2.13-10.1.1.jar'),
 ('

In [6]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext, functions as F
from pyspark.sql.functions import *

mongo_uri = "mongodb://hadoop-vm.internal.cloudapp.net:27017/ca2"

# create a spark session
spark = SparkSession \
    .builder \
    .appName("Tweets") \
    .config("spark.driver.memory", "15g") \
    .config("spark.mongodb.read.connection.uri", mongo_uri) \
    .config("spark.mongodb.write.connection.uri", mongo_uri) \
    #.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.1.1') \
    .getOrCreate()


# read data from mongodb collection "questions" into a dataframe "df"
df = spark.read \
    .format("mongodb") \
    .option("uri", mongo_uri) \
    .option("database", "ca2") \
    .option("collection", "tweets") \
    .load()

df.printSchema()

Py4JJavaError: An error occurred while calling o138.load.
: java.lang.NoClassDefFoundError: org/bson/BsonValue
	at com.mongodb.spark.sql.connector.MongoTableProvider.inferSchema(MongoTableProvider.java:62)
	at org.apache.spark.sql.execution.datasources.v2.DataSourceV2Utils$.getTableFromProvider(DataSourceV2Utils.scala:81)
	at org.apache.spark.sql.DataFrameReader.$anonfun$load$1(DataFrameReader.scala:233)
	at scala.Option.map(Option.scala:230)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:210)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:174)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvo

{"exception":{"exception_class":"java.lang.IllegalArgumentException","exception_message":"Too large frame: 1586112597084667896","stacktrace":"java.lang.IllegalArgumentException: Too large frame: 1586112597084667896\n\tat org.sparkproject.guava.base.Preconditions.checkArgument(Preconditions.java:119)\n\tat org.apache.spark.network.util.TransportFrameDecoder.decodeNext(TransportFrameDecoder.java:148)\n\tat org.apache.spark.network.util.TransportFrameDecoder.channelRead(TransportFrameDecoder.java:98)\n\tat io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:379)\n\tat io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:365)\n\tat io.netty.channel.AbstractChannelHandlerContext.fireChannelRead(AbstractChannelHandlerContext.java:357)\n\tat io.netty.channel.DefaultChannelPipeline$HeadContext.channelRead(DefaultChannelPipeline.java:1410)\n\tat io.netty.channel.AbstractChannelHandlerContext.invokeCha

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 34838)
Traceback (most recent call last):
  File "/anaconda/envs/py38_default/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/anaconda/envs/py38_default/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/anaconda/envs/py38_default/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/anaconda/envs/py38_default/lib/python3.8/socketserver.py", line 747, in __init__
    self.handle()
  File "/dsvm/tools/spark/current/python/pyspark/accumulators.py", line 260, in handle
    poll(authenticate_and_accum_updates)
  File "/dsvm/tools/spark/current/python/pyspark/accumulators.py", line 235, in poll
    if func():
  File "/dsvm/tools/spark/current/python/pyspark/accumula

{"exception":{"exception_class":"java.lang.IllegalArgumentException","exception_message":"Too large frame: 1586112597084667896","stacktrace":"java.lang.IllegalArgumentException: Too large frame: 1586112597084667896\n\tat org.sparkproject.guava.base.Preconditions.checkArgument(Preconditions.java:119)\n\tat org.apache.spark.network.util.TransportFrameDecoder.decodeNext(TransportFrameDecoder.java:148)\n\tat org.apache.spark.network.util.TransportFrameDecoder.channelRead(TransportFrameDecoder.java:98)\n\tat io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:379)\n\tat io.netty.channel.AbstractChannelHandlerContext.invokeChannelRead(AbstractChannelHandlerContext.java:365)\n\tat io.netty.channel.AbstractChannelHandlerContext.fireChannelRead(AbstractChannelHandlerContext.java:357)\n\tat io.netty.channel.DefaultChannelPipeline$HeadContext.channelRead(DefaultChannelPipeline.java:1410)\n\tat io.netty.channel.AbstractChannelHandlerContext.invokeCha